<a href="https://colab.research.google.com/github/Ashitha11/Prompt-engineering-course/blob/main/VectorDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00


In [2]:
pip install python-dotenv

In [3]:
from google.colab import drive              #access to drive
drive.mount('/content/drive')

import os
from dotenv import load_dotenv

# Define the path to your .env file inside Google Drive
env_path = "/content/drive/My Drive/Colab/.env"  # Adjust if needed

# Load environment variables from the file
load_dotenv(env_path)

# Access the API key
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

Mounted at /content/drive


In [4]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.8 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [6]:
!mkdir pdfs

In [18]:
loader=PyPDFDirectoryLoader("pdfs")
data = loader.load()                    #pdf data is stored in 'data' in list form
#loader.load() --> would display the pdf data

In [19]:
#tokenisation - divides data into tokens (list form)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)      #each chunk 500 characters long 20 characters of one chunk overlaps with the next chunk
text_chunks = text_splitter.split_documents(data)


In [20]:
#creating an object of OpenAIEmbeddings
embedding = OpenAIEmbeddings()
#embedding.embed_query("Hello") --> displays the embedded vector
#len(embedding.embed_query("Hello")) --> displays length of embedded vector

#next step --> the created embedded vector to be stored in pinecone

In [21]:
import os
from pinecone import Pinecone, ServerlessSpec

api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)


In [22]:
#initializing index name
index_name = "testing"

**CREATE EMBEDDINGS FOR EACH TEXT CHUNK**

In [24]:
from langchain.vectorstores import Pinecone as LangchainPinecone # Import Pinecone from langchain.vectorstores

docsearch = LangchainPinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name) # Use LangchainPinecone instead of Pinecone


# This converts text into searchable vector embeddings and stores them in a Pinecone index for fast similarity search.


In [25]:
docsearch

In [26]:
query = "YOLOv7 outperforms which models"
#similarity search
docs = docsearch.similarity_search(query)

In [27]:
docs                          #this doesnt give us the exact answer we need to the query ; it is in the form of vector

[Document(metadata={}, page_content='be superior in quality while being more parallelizable and requiring signiﬁcantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\nto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new single-model state-of-the-art BLEU score of 41.0 after\ntraining for 3.5 days on eight GPUs, a small fraction of the training costs of the'),
 Document(metadata={}, page_content='the competitive models.\nOn the WMT 2014 English-to-French translation task, our big model achieves a BLEU score of 41.0,\noutperforming all of the previously published single models, at less than 1/4 the training cost of the\nprevious state-of-the-art model. The Transformer (big) model trained for English-to-French used\ndropout rate Pdrop = 0.1, instead of 0.3.\nFor the base models, we used a single model obtained by averaging the 

In [32]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
llm = OpenAI()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
#It creates a question-answering system that retrieves relevant documents from Pinecone and uses an LLM (like GPT) to generate answers.

In [34]:
query = "Explain attention function"
qa.run(query)

' The attention function is a way of mapping a query and a set of key-value pairs to an output, using vectors. It computes a weighted sum of the values, with the weights determined by the compatibility between the query and each key. This mechanism, known as self-attention or intra-attention, has been used successfully in various natural language processing tasks. Two commonly used attention functions are additive attention and dot-product attention, with the latter being identical to the algorithm described in section 3.2 except for a scaling factor. The use of masking and offset output embeddings ensures that the predictions for each position only depend on the known outputs at earlier positions.'

In [41]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == "exit":
    print("Exiting the program.")
    sys.exit()
  elif user_input == "":
    continue
  result=qa({'query':user_input})
  print(f"Answer: {result['result']}")
  break

Input Prompt: can you summarize this pdf in 100 words. bullet points
Answer:  

- This paper discusses a new model called "Attention Is All You Need" for machine translation.
- The authors are from Google Brain, Google Research, and the University of Toronto.
- They trained their models on one machine with 8 NVIDIA P100 GPUs for 12 hours.
- They used beam search and length penalty for inference.
- Their results and training costs were compared to other models.
- They also experimented with different hyperparameters on a development set.
- The paper includes a list of references for related work.
